In [1]:
'''
extract features for all images in out dataset(include valid set, test set)
by doing this, you can reduce for image feature extraction
'''

'\nextract features for all images in out dataset(include valid set, test set)\nby doing this, you can reduce for image feature extraction\n'

In [2]:
import os
import pickle
import pprint
import numpy as np
import time
import gzip
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model


In [3]:
start = time.time()
model = VGG16()
model = Model(inputs=model.inputs, outputs=model.layers[-2].output) # drop last prediction layer
end = time.time()
print('Elapsed:', end-start, 'sec')

Elapsed: 2.6946074962615967 sec


In [4]:
start = time.time()
image_dir = os.getcwd() + '/Flicker8k_Dataset'
features = dict()
for i, filename in enumerate(os.listdir(image_dir)):
    filepath = image_dir + '/' + filename
    image = load_img(filepath, target_size=(224, 224, 3)) # VGG16 input size
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0) # according to keras document
    image = preprocess_input(image) # according to keras document
    feature = model.predict(image)
    image_id = filename.split('.')[0]
    features[image_id] = feature
    print('{0:>4}/{1:<4}-->{2}'.format(i, len(os.listdir(image_dir)), image_id), end='\r')

In [5]:
print('# of images feature extracted:', len(features))
end = time.time()
print('Elapsed time: {}sec'.format(end-start))

# of images feature extracted: 8091
Elapsed time: 1673.702288866043sec


In [6]:
total_dataset = set([key for key in features])
train_valid_test_set = set()
train_info_filename = os.getcwd() + '/Flickr8k_text/Flickr_8k.trainImages.txt'
valid_info_filename = os.getcwd() + '/Flickr8k_text/Flickr_8k.devImages.txt'
test_info_filename = os.getcwd() + '/Flickr8k_text/Flickr_8k.testImages.txt'

with open(train_info_filename, 'r') as f:
    text = f.read()
    image_filenames = text.split('\n') # get image filenames in dataset: (train, valid, test)
    image_ids = [x.split('.')[0] for x in image_filenames if x] # remove '.jpg', igore empty filename
    train_valid_test_set.update(image_ids)

with open(valid_info_filename, 'r') as f:
    text = f.read()
    image_filenames = text.split('\n') # get image filenames in dataset: (train, valid, test)
    image_ids = [x.split('.')[0] for x in image_filenames if x] # remove '.jpg', igore empty filename
    train_valid_test_set.update(image_ids)
                                
with open(test_info_filename, 'r') as f:
    text = f.read()
    image_filenames = text.split('\n') # get image filenames in dataset: (train, valid, test)
    image_ids = [x.split('.')[0] for x in image_filenames if x] # remove '.jpg', igore empty filename
    train_valid_test_set.update(image_ids)
                                
# There are unused images
print('len total imagestotal_dataset_dataset:', len(total_dataset))
print('len train valid test images:', len(train_valid_test_set))

len total imagestotal_dataset_dataset: 8091
len train valid test images: 8000


In [7]:
start = time.time()
with gzip.open('image_features.pkl.zip', 'wb') as f:
    pickle.dump(features, f)
    
end = time.time()
print('Elapsed:', end-start, 'sec')

Elapsed: 70.21755266189575 sec
